In [1]:
import numpy as np
from precommit_analysis.keras_mnist_example import prepare_data, create_mnist_cnn_model
from precommit_analysis.generators import sparse_mnist_generator_nonzero, eval_generator

batch_size = 128
num_classes = 10
epochs = 12

/home/hrosspet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
x_train, y_train, x_test, y_test, input_shape = prepare_data(num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Train the judge

- sparse MNIST classifier
- 6 non-zero pixels are randomly sampled from an input image

In [7]:
training_data_generator = sparse_mnist_generator_nonzero(x_train, y_train, batch_size, sparsity=6)
val_data_generator = sparse_mnist_generator_nonzero(
    x_test,
    y_test,
    batch_size=x_test.shape[0],
    sparsity=6,
    shuffle=False
)
val_data = next(val_data_generator)

### First train a judge on only 5k batches

In [68]:
judge = create_mnist_cnn_model(num_classes, input_shape)
judge.fit_generator(training_data_generator,
          steps_per_epoch=1000,
          epochs=5,
          verbose=1,
          validation_data=val_data)

Epoch 1/5
1000/1000 [==============================] - 256s 256ms/step - loss: 1.6545 - acc: 0.4302 - val_loss: 1.4303 - val_acc: 0.5097
Epoch 2/5
1000/1000 [==============================] - 254s 254ms/step - loss: 1.5218 - acc: 0.4699 - val_loss: 1.3892 - val_acc: 0.5206
Epoch 3/5
1000/1000 [==============================] - 270s 270ms/step - loss: 1.4913 - acc: 0.4799 - val_loss: 1.3654 - val_acc: 0.5261
Epoch 4/5
1000/1000 [==============================] - 261s 261ms/step - loss: 1.4786 - acc: 0.4850 - val_loss: 1.3547 - val_acc: 0.5333
Epoch 5/5
1000/1000 [==============================] - 261s 261ms/step - loss: 1.4656 - acc: 0.4898 - val_loss: 1.3435 - val_acc: 0.5344


In [70]:
judge.save('model_sparse_mnist_generator_nonzero_5k.h5py')

### Train a better judge on 30k batches

In [9]:
judge = create_mnist_cnn_model(num_classes, input_shape)
judge.fit_generator(training_data_generator,
          steps_per_epoch=1000,
          epochs=30,
          verbose=1,
          validation_data=val_data)

Epoch 1/30
1000/1000 [==============================] - 268s 268ms/step - loss: 1.6560 - acc: 0.4282 - val_loss: 1.4424 - val_acc: 0.4989
Epoch 2/30
1000/1000 [==============================] - 270s 270ms/step - loss: 1.5235 - acc: 0.4681 - val_loss: 1.3988 - val_acc: 0.5193
Epoch 3/30
1000/1000 [==============================] - 266s 266ms/step - loss: 1.4985 - acc: 0.4776 - val_loss: 1.3772 - val_acc: 0.5220
Epoch 4/30
1000/1000 [==============================] - 259s 259ms/step - loss: 1.4784 - acc: 0.4843 - val_loss: 1.3614 - val_acc: 0.5286
Epoch 5/30
1000/1000 [==============================] - 253s 253ms/step - loss: 1.4679 - acc: 0.4893 - val_loss: 1.3561 - val_acc: 0.5306
Epoch 6/30
1000/1000 [==============================] - 259s 259ms/step - loss: 1.4545 - acc: 0.4950 - val_loss: 1.3465 - val_acc: 0.5325
Epoch 7/30
1000/1000 [==============================] - 253s 253ms/step - loss: 1.4550 - acc: 0.4943 - val_loss: 1.3363 - val_acc: 0.5359
Epoch 8/30
1000/1000 [============

In [10]:
judge.save('model_sparse_mnist_generator_nonzero_30k.h5py')